In [1]:
from image_pose_transfer import (
    create_model,
    load_state_dict,
    center_crop_to_512,
    center_crop_pose_to_512,
    get_cond_control,
    save_image,
    openpose,
)
import torch

/home/slohiya3/.conda/envs/magicpose/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


cuda
cuda
cuda


In [2]:
class objectview(object):
    def __init__(self, d):
        self.__dict__ = d
args = objectview({
    "model_config": "model_lib/ControlNet/models/cldm_v15_reference_only_pose.yaml",
    "image_pretrain_dir": "./pretrained_weights/model_state-110000.th",
    "input_image_path": "./example_data/image/out-of-domain/001.png",
    # "input_image_path": "./example_data/image/out-of-domain/181020.png",
    "pose_map_path": "./example_data/pose_sequence/001/0001.png",
    "output_image_path": "output_image.jpg",
    "eta": 0.0,
    "device": "cuda",
    "control_type": "body+hand+face",
    "wonoise": True,
    "text_prompt": None,
    "extract_pose": True
})

In [3]:
# Load the pre-trained model
model = create_model(args.model_config).cpu()
model.to(args.device)
load_state_dict(model, args.image_pretrain_dir, strict=True)

ControlLDMReferenceOnlyPose: Running in eps-prediction mode
Setting up MemoryEfficientCrossAttention. Query dim is 320, context_dim is None and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 320, context_dim is 768 and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 320, context_dim is None and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 320, context_dim is 768 and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 640, context_dim is None and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 640, context_dim is 768 and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 640, context_dim is None and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 640, context_dim is 768 and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 1280, context_dim is None and using 8 heads.
Setting up MemoryEfficientCrossAttention. Query dim 

Some weights of the model checkpoint at openai/clip-vit-large-patch14 were not used when initializing CLIPTextModel: ['vision_model.encoder.layers.3.layer_norm2.weight', 'vision_model.encoder.layers.23.layer_norm2.bias', 'vision_model.encoder.layers.13.layer_norm2.bias', 'vision_model.encoder.layers.12.self_attn.out_proj.bias', 'vision_model.encoder.layers.1.self_attn.v_proj.bias', 'vision_model.encoder.layers.18.self_attn.q_proj.bias', 'logit_scale', 'vision_model.encoder.layers.17.self_attn.v_proj.weight', 'vision_model.encoder.layers.3.self_attn.q_proj.weight', 'vision_model.encoder.layers.9.self_attn.k_proj.weight', 'vision_model.encoder.layers.20.self_attn.out_proj.bias', 'vision_model.pre_layrnorm.weight', 'vision_model.encoder.layers.14.self_attn.v_proj.weight', 'vision_model.encoder.layers.13.self_attn.out_proj.weight', 'vision_model.encoder.layers.19.layer_norm1.bias', 'vision_model.encoder.layers.13.self_attn.q_proj.weight', 'vision_model.encoder.layers.23.mlp.fc2.bias', 'vis

()
{'linear_start': 0.00085, 'linear_end': 0.012, 'num_timesteps_cond': 1, 'log_every_t': 200, 'timesteps': 1000, 'first_stage_key': 'jpg', 'cond_stage_key': 'txt', 'image_size': 64, 'channels': 4, 'cond_stage_trainable': False, 'conditioning_key': 'crossattn', 'monitor': 'val/loss_simple_ema', 'scale_factor': 0.18215, 'use_ema': False, 'unet_config': {'target': 'model_lib.ControlNet.cldm.cldm.ControlledUnetModelAttnPose', 'params': {'image_size': 32, 'in_channels': 4, 'out_channels': 4, 'model_channels': 320, 'attention_resolutions': [4, 2, 1], 'num_res_blocks': 2, 'channel_mult': [1, 2, 4, 4], 'num_heads': 8, 'use_spatial_transformer': True, 'transformer_depth': 1, 'context_dim': 768, 'use_checkpoint': True, 'legacy': False}}, 'first_stage_config': {'target': 'model_lib.ControlNet.ldm.models.autoencoder.AutoencoderKL', 'params': {'embed_dim': 4, 'monitor': 'val/rec_loss', 'ddconfig': {'double_z': True, 'z_channels': 4, 'resolution': 256, 'in_channels': 3, 'out_ch': 3, 'ch': 128, 'ch_

In [4]:
# Load the input image and pose map
input_image = center_crop_to_512(args.input_image_path).unsqueeze(0).to(args.device)
pose_map = center_crop_pose_to_512(args.pose_map_path,
                                   extract_pose_from_image=args.extract_pose
                                   ).unsqueeze(0).to(args.device)

# Get the control maps
c_cat_list, cond_img_cat = get_cond_control(args, input_image, pose_map, args.device, model=model,)

model.eval()

text = [""] if args.text_prompt is None else [args.text_prompt]
c_cross = model.get_learned_conditioning(text)
uc_cross = model.get_unconditional_conditioning(1)

pose_input = c_cat_list[0]

c = {"c_concat": [pose_input], "c_crossattn": [c_cross], "image_control": cond_img_cat, "wonoise": args.wonoise, "overlap_sampling": False}
uc = {"c_concat": [pose_input], "c_crossattn": [uc_cross], "wonoise": args.wonoise, "overlap_sampling": False}

noise_shape = (1, model.channels, model.image_size, model.image_size)
noise = torch.randn(noise_shape).cuda()

with torch.no_grad():
    output_image, _ = model.sample_log(
        cond=c,
        batch_size=1,
        ddim=True,
        ddim_steps=50, eta=args.eta,
        unconditional_guidance_scale=7,
        unconditional_conditioning=uc,
        inpaint=None,
        x_T=noise,                                        
    )

    output_image = model.decode_first_stage(output_image)

# Save the output image
save_image(output_image.clamp(-1, 1).cpu().add(1).mul(0.5), args.output_image_path)

Data shape for DDIM sampling is (1, 4, 64, 64), eta 0.0
Running DDIM Sampling with 50 timesteps


DDIM Sampler: 100%|██████████| 50/50 [00:15<00:00,  3.13it/s]
